In [5]:
from datetime import date, datetime
import pandas as pd
import numpy as np
import yaml
from sqlalchemy import create_engine, inspect, text

In [2]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_bodega = config['bodega']

# Crear conexión a la bodega
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"
bodega_conn = create_engine(url_bodega)

In [3]:
# Crear datos para la dimensión fecha
fecha_inicio = '2023-01-01'
fecha_fin = '2024-12-31'

In [6]:
# Generar fechas
fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')
dim_fecha = pd.DataFrame({'fecha': fechas})

# Convertir a timestamp sin hora (solo fecha)
dim_fecha['fecha'] = dim_fecha['fecha'].dt.strftime('%Y-%m-%d')
dim_fecha['fecha'] = pd.to_datetime(dim_fecha['fecha'])

In [7]:
# Extraer componentes de la fecha
dim_fecha['año'] = dim_fecha['fecha'].dt.year
dim_fecha['mes'] = dim_fecha['fecha'].dt.month
dim_fecha['dia'] = dim_fecha['fecha'].dt.day
dim_fecha['dia_semana'] = dim_fecha['fecha'].dt.dayofweek  # 0 = Lunes, 6 = Domingo

In [8]:
# Agregar fecha de carga
dim_fecha['saved'] = date.today()

In [9]:
# Verificar los datos
dim_fecha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   fecha       731 non-null    datetime64[ns]
 1   año         731 non-null    int32         
 2   mes         731 non-null    int32         
 3   dia         731 non-null    int32         
 4   dia_semana  731 non-null    int32         
 5   saved       731 non-null    object        
dtypes: datetime64[ns](1), int32(4), object(1)
memory usage: 23.0+ KB


In [10]:
print("\nMuestra de los datos:")
print(dim_fecha.head())


Muestra de los datos:
       fecha   año  mes  dia  dia_semana       saved
0 2023-01-01  2023    1    1           6  2024-11-07
1 2023-01-02  2023    1    2           0  2024-11-07
2 2023-01-03  2023    1    3           1  2024-11-07
3 2023-01-04  2023    1    4           2  2024-11-07
4 2023-01-05  2023    1    5           3  2024-11-07


In [11]:
# Cargar en la bodega
dim_fecha.to_sql('dim_fecha', bodega_conn, if_exists='replace', index_label='key_dim_fecha')

731